In [1]:
import pandas as pd

data = pd.read_csv("./data/extraction_results.csv")
data = data[data["prompt"] == 5]
data

,Тема,Социальная сеть,Целевая аудитория,Дополнительная информация,num_message,stage,prompt,model
200,Список книг для расширения кругозора и поиска ...,ВКонтакте,"Любители литературы, интересующиеся саморазвит...",Предложить список из 5-7 книг разных жанров с ...,0,extract,5,gemma-2-9b-it-Q8_0_L
201,Сколько книг нужно читать в год?,Telegram,"Любой пользователь, интересующийся саморазвити...",Поделиться личным опытом чтения. Дать советы п...,1,extract,5,gemma-2-9b-it-Q8_0_L
202,Топ-5 упражнений для утренней зарядки,Instagram,"Люди, интересующиеся здоровым образом жизни и ...",Использовать визуальный контент: видео или фот...,2,extract,5,gemma-2-9b-it-Q8_0_L
203,История успеха известного спортсмена: преодоле...,ВКонтакте,"Люди всех возрастов, интересующиеся спортом и ...","Рассказать о жизненном пути спортсмена, его пр...",3,extract,5,gemma-2-9b-it-Q8_0_L
204,Как правильно обрезать плодовые деревья весной...,Telegram,Садоводы и огородники,NaN,4,extract,5,gemma-2-9b-it-Q8_0_L
...,...,...,...,...,...,...,...,...
715,Советы по созданию эффективной команды,Telegram,"Работники и руководители, интересующиеся вопро...",Использовать формат чата для обмена советами и...,35,extract,5,suzume-llama-3-8b-Q8_0
716,Влияние окружающей среды на здоровье,ВКонтакте,"Люди всех возрастов, интересующиеся вопросами ...",Использовать ссылки на научные исследования и ...,36,extract,5,suzume-llama-3-8b-Q8_0
717,Как развить навыки общения,Telegram,"Люди всех возрастов, заинтересованные в улучше...",Использовать формат чата для обмена советами и...,37,extract,5,suzume-llama-3-8b-Q8_0
718,Влияние путешествий на личность,Twitter,Люди любящие путешествовать и интересующиеся в...,Использовать хэштеги #путешествия #личностноер...,38,extract,5,suzume-llama-3-8b-Q8_0


In [2]:
import os

def get_models(directory, extension=".gguf"):
    return [filename.split(".")[0] for filename in os.listdir(directory) if filename.endswith(extension)]

models = get_models("../models")
print(models)

['gemma-2-9b-it-Q8_0_L', 'saiga-llama-3-q8_0', 'suzume-llama-3-8b-Q8_0']


In [3]:
from langchain_community.llms import LlamaCpp
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


llama3_tokens = {
    'first_token': '<|begin_of_text|>',
    'start_header': '<|start_header_id|>', 
    'end_header': '<|end_header_id|>', 
    'end_message': '<|eot_id|>'
}
gemma2_tokens = {
    'first_token': '',
    'start_header': '<start_of_turn>',
    'end_header': '',
    'end_message': '<end_of_turn>'
}

def get_llm(path):

    model = LlamaCpp(
        model_path=path,
        temperature=0,
        n_ctx=8192,
        max_tokens=2000,
        top_p=1,
        repeat_penalty=1.1,
        # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        verbose=False,
        n_gpu_layers=33
    )
    return model

def get_chat(model_name):
    chat = ChatOllama(model=model_name, temperature=0)
    return chat

In [4]:
web_search_prompt_base = """{first_token}{start_header}system{end_header}

Ты являешься экспертом в составлении поисковых запросов в Интернете для поиска информации.
Твоя задача на основе темы, целевой аудитории и дополнительной информации составить от 1 до 2 поисковых запросов с целью раскрытия выбранной темы.
Верни JSON с помощью одного ключа «queries» без преамбулы или объяснения.

Пример:
{{
    «queries» : [«Текст запроса 1», «Текст запроса 2»]
}}

Информация для составления запроса:
Тема: {topic}
Целевая аудитория: {audience}
Дополнительная информация: {additional_information}

{end_message}{start_header}assistant{end_header}\n"""

web_search_prompt_candidate_1 = """{first_token}{start_header}system{end_header}

Ты — эксперт в составлении интернет-поисковых запросов. Твоя задача — на основе заданной темы, целевой аудитории и дополнительной информации составить от 1 до 2 релевантных поисковых запросов, чтобы максимально полно раскрыть тему.
Верни JSON с ключом «queries», без объяснений.

Пример:
{{
    «queries»: [«Текст запроса 1», «Текст запроса 2»]
}}

Информация для составления запроса:
Тема: {topic}
Целевая аудитория: {audience}
Дополнительная информация: {additional_information}

{end_message}{start_header}assistant{end_header}\n"""
web_search_prompt_candidate_2 = """{first_token}{start_header}system{end_header}

Ты — эксперт в составлении интернет-поисковых запросов. Твоя задача — на основе заданной темы, целевой аудитории и дополнительной информации составить от 1 до 2 релевантных поисковых запросов, чтобы максимально полно раскрыть тему.

Требования к запросам:
- Используй точные и релевантные ключевые слова.
- Формат запросов может быть как вопросительным, так и утвердительным.
- Учитывай контекст и уровень знаний целевой аудитории.
- Используй язык на котором написан контекст.

Пример:
{{
    «queries»: [«Текст запроса 1», «Текст запроса 2»]
}}

Верни JSON с ключом «queries», без объяснений.

Информация для составления запроса:
Тема: {topic}
Целевая аудитория: {audience}
Дополнительная информация: {additional_information}

{end_message}{start_header}assistant{end_header}\n"""
web_search_prompt_candidate_3 = """{first_token}{start_header}system{end_header}

Ты — эксперт в составлении интернет-поисковых запросов. На основе заданной темы, целевой аудитории и дополнительной информации составь от 1 до 2 релевантных поисковых запросов, чтобы максимально полно раскрыть тему.

Требования к запросам:
- Используй точные и релевантные ключевые слова.
- Формат запросов может быть как вопросительным, так и утвердительным.
- Учитывай контекст и уровень знаний целевой аудитории.
- Используй язык на котором написан контекст.

Пример для конкретной задачи:
Тема: "Технологии в образовании"
Целевая аудитория: Учителя средней школы
Дополнительная информация: Обзор применяемых технологий в образовании

{{
    «queries»: ["Современные технологии в образовании", "Лучшие образовательные технологии для учителей"]
}}

Верни JSON с ключом «queries», без объяснений.

Информация для составления запроса:
Тема: {topic}
Целевая аудитория: {audience}
Дополнительная информация: {additional_information}

{end_message}{start_header}assistant{end_header}\n"""

web_serach_candidates = [
    web_search_prompt_base,
    web_search_prompt_candidate_1,
    web_search_prompt_candidate_2,
    web_search_prompt_candidate_3
]

In [5]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate

import pandas as pd
import tqdm
from langchain_core.exceptions import OutputParserException


def get_web_search_prompt(prompt):
    web_search_prompt = PromptTemplate(
        template=prompt,
        input_variables=["first_token", "start_header", "end_header", "end_message", "topic", "audience", "additional_information"],
    )
    return web_search_prompt

In [6]:
results = pd.DataFrame()


for model in get_models("../models"):
    print(model)
    llm = get_chat(model)
    input_dict = gemma2_tokens.copy() if "gemma" in model else llama3_tokens.copy()
    for candidate in tqdm.tqdm(web_serach_candidates, total=len(web_serach_candidates)):
        web_search_prompt = get_web_search_prompt(candidate)
        web_search_chain = web_search_prompt | llm | JsonOutputParser()
        outputs = {}
        inputs = data[data['model'] == models[0]].reset_index(drop=True)
        for num, row in inputs.iterrows():
            input_dict["topic"] = row["Тема"]
            input_dict["audience"] = row["Целевая аудитория"]
            input_dict["additional_information"] = row["Дополнительная информация"]
            try:
                output = web_search_chain.invoke(input_dict)
            except OutputParserException:
                output = {}
            output = {key: output.get(key, None) for key in ["queries"]}
 
            output["num_message"] = num
            for key in output.keys():
                outputs[key] = outputs.get(key, []) + [output[key]]

        extract_df = pd.DataFrame.from_dict(outputs)
        
        extract_df['stage'] = "web_search"
        extract_df['prompt'] = web_serach_candidates.index(candidate)
        extract_df['model'] = model.split("/")[-1]
        results = pd.concat([results, extract_df], axis=0)
        results.to_csv('./data/web_search_results.csv', index=False)

gemma-2-9b-it-Q8_0_L


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [08:19<00:00, 124.94s/it]


saiga-llama-3-q8_0


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [07:29<00:00, 112.45s/it]


suzume-llama-3-8b-Q8_0


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [07:46<00:00, 116.52s/it]


In [21]:
results["query_counts"] = results["queries"].apply(lambda x: len(x) if isinstance(x, list) else 0)
results["query_counts"].value_counts()

query_counts
2    437
0     43
Name: count, dtype: int64

In [23]:
results[results["query_counts"] == 0]

,queries,num_message,stage,prompt,model,query_counts
12,None,12,web_search,0,saiga-llama-3-q8_0,0
6,None,6,web_search,1,saiga-llama-3-q8_0,0
24,None,24,web_search,1,saiga-llama-3-q8_0,0
0,None,0,web_search,3,saiga-llama-3-q8_0,0
1,None,1,web_search,3,saiga-llama-3-q8_0,0
2,None,2,web_search,3,saiga-llama-3-q8_0,0
3,None,3,web_search,3,saiga-llama-3-q8_0,0
4,None,4,web_search,3,saiga-llama-3-q8_0,0
5,None,5,web_search,3,saiga-llama-3-q8_0,0
6,None,6,web_search,3,saiga-llama-3-q8_0,0


In [22]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["query_counts", "queries"]].count() / \
    (results.shape[0] / len(web_serach_candidates) / len(models))

query_counts  queries
prompt model                                        
0      gemma-2-9b-it-Q8_0_L             1.0    1.000
       saiga-llama-3-q8_0               1.0    0.975
       suzume-llama-3-8b-Q8_0           1.0    1.000
1      gemma-2-9b-it-Q8_0_L             1.0    1.000
       saiga-llama-3-q8_0               1.0    0.950
       suzume-llama-3-8b-Q8_0           1.0    1.000
2      gemma-2-9b-it-Q8_0_L             1.0    1.000
       saiga-llama-3-q8_0               1.0    1.000
       suzume-llama-3-8b-Q8_0           1.0    1.000
3      gemma-2-9b-it-Q8_0_L             1.0    1.000
       saiga-llama-3-q8_0               1.0    0.000
       suzume-llama-3-8b-Q8_0           1.0    1.000

In [24]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["query_counts"]].mean() 

query_counts
prompt model                               
0      gemma-2-9b-it-Q8_0_L            2.00
       saiga-llama-3-q8_0              1.95
       suzume-llama-3-8b-Q8_0          2.00
1      gemma-2-9b-it-Q8_0_L            2.00
       saiga-llama-3-q8_0              1.90
       suzume-llama-3-8b-Q8_0          2.00
2      gemma-2-9b-it-Q8_0_L            2.00
       saiga-llama-3-q8_0              2.00
       suzume-llama-3-8b-Q8_0          2.00
3      gemma-2-9b-it-Q8_0_L            2.00
       saiga-llama-3-q8_0              0.00
       suzume-llama-3-8b-Q8_0          2.00

In [25]:
import nest_asyncio
nest_asyncio.apply()

In [30]:
results = results[results["prompt"] != 3]

In [54]:
from tqdm import tqdm
from ragas.metrics import (
    answer_similarity
)
from ragas import evaluate
from langchain_community.embeddings import OllamaEmbeddings
from datasets import Dataset

metrics = pd.DataFrame(
    columns=["answer_similarity"])

critic_model = "gemma-2-27b-it-Q6_K_L"
langchain_llm = ChatOllama(model=critic_model, temperature=0)
langchain_embeddings = OllamaEmbeddings(model=critic_model)

for idx, row in tqdm(results.iterrows(), total=len(results)):

    if  isinstance(row["queries"], type(None)):
        metrics = pd.concat([metrics, pd.DataFrame([{"answer_similarity": 0.0}])], axis=0)
        continue
    
    sample = data[(data.model == row.model) & (data.num_message == row.num_message)]
    message = f"Информация для составления запроса:\nТема: {sample['Тема'].iloc[0]}\nЦелевая аудитория: {sample['Целевая аудитория'].iloc[0]}\
    \nДополнительная информация: {sample['Дополнительная информация'].iloc[0]}"

    sample = pd.DataFrame().from_dict({k: v if k in ["queries"] else [v] * row.query_counts for k, v in row.to_dict().items()})
    sample["ground_truth"] = message
    sample.rename(columns={"queries": "answer"}, inplace=True)

    r = evaluate(
        Dataset.from_pandas(sample[['answer', 'ground_truth']]),
        metrics=[answer_similarity],
        llm=langchain_llm,
        embeddings=langchain_embeddings,
        raise_exceptions=False
    )

    r = dict(r)
    metrics = pd.concat([metrics, pd.DataFrame([r])], axis=0)

  0%|                                                                                          | 0/228 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

  1%|▋                                                                                 | 2/228 [00:07<14:55,  3.96s/it]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

  1%|█                                                                                 | 3/228 [00:12<16:26,  4.39s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 226/228 [18:09<00:09,  4.82s/it]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 227/228 [18:14<00:04,  4.80s/it]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 228/228 [18:18<00:00,  4.82s/it]


In [56]:
metrics

,answer_similarity
0,0.706136
0,0.811711
0,0.817909
0,0.716549
0,0.574703
...,...
0,0.707698
0,0.747172
0,0.807445
0,0.719724


In [57]:
results = pd.concat([results.reset_index(drop=True), metrics.reset_index(drop=True)], axis=1)
results

,queries,num_message,stage,prompt,model,query_counts,answer_similarity
0,[книги для расширения кругозора и вдохновения ...,0,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.706136
1,"[сколько книг читать в год для саморазвития, к...",1,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.811711
2,"[топ 5 упражнений для утренней зарядки видео, ...",2,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.817909
3,[история успеха известного спортсмена преодоле...,3,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.716549
4,[правила весенней обрезки плодовых деревьев дл...,4,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.574703
...,...,...,...,...,...,...,...
355,"[Как создать эффективную команду для работы, П...",35,web_search,2,suzume-llama-3-8b-Q8_0,2,0.707698
356,[Как окружающая среда влияет на моё физическое...,36,web_search,2,suzume-llama-3-8b-Q8_0,2,0.747172
357,"[Как развить навыки общения?, Методы для улучш...",37,web_search,2,suzume-llama-3-8b-Q8_0,2,0.807445
358,"[Как путешествия влияют на личность?, Путешест...",38,web_search,2,suzume-llama-3-8b-Q8_0,2,0.719724


In [60]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["queries", "query_counts", "answer_similarity"]].count() / \
    (results.shape[0] / (len(web_serach_candidates) -1 ) / len(models))

queries  query_counts  answer_similarity
prompt model                                                           
0      gemma-2-9b-it-Q8_0_L      1.000           1.0                1.0
       saiga-llama-3-q8_0        0.975           1.0                1.0
       suzume-llama-3-8b-Q8_0    1.000           1.0                1.0
1      gemma-2-9b-it-Q8_0_L      1.000           1.0                1.0
       saiga-llama-3-q8_0        0.950           1.0                1.0
       suzume-llama-3-8b-Q8_0    1.000           1.0                1.0
2      gemma-2-9b-it-Q8_0_L      1.000           1.0                1.0
       saiga-llama-3-q8_0        1.000           1.0                1.0
       suzume-llama-3-8b-Q8_0    1.000           1.0                1.0

In [61]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["query_counts", "answer_similarity"]].mean()

query_counts  answer_similarity
prompt model                                                  
0      gemma-2-9b-it-Q8_0_L            2.00           0.733937
       saiga-llama-3-q8_0              1.95           0.803451
       suzume-llama-3-8b-Q8_0          2.00           0.779369
1      gemma-2-9b-it-Q8_0_L            2.00           0.764456
       saiga-llama-3-q8_0              1.90           0.795817
       suzume-llama-3-8b-Q8_0          2.00           0.783453
2      gemma-2-9b-it-Q8_0_L            2.00           0.755251
       saiga-llama-3-q8_0              2.00           0.809786
       suzume-llama-3-8b-Q8_0          2.00           0.762233

In [62]:
results.set_index(['prompt']).groupby(level=['prompt'])[
    ["query_counts", "answer_similarity"]].mean()

,query_counts,answer_similarity
prompt,,
0,1.983333,0.772252
1,1.966667,0.781242
2,2.000000,0.775757


In [63]:
results.set_index(['model']).groupby(level=['model'])[
    ["query_counts", "answer_similarity"]].mean()

,query_counts,answer_similarity
model,,
gemma-2-9b-it-Q8_0_L,2.00,0.751214
saiga-llama-3-q8_0,1.95,0.803018
suzume-llama-3-8b-Q8_0,2.00,0.775018


In [69]:
data[data["num_message"] == 32]

,Тема,Социальная сеть,Целевая аудитория,Дополнительная информация,num_message,stage,prompt,model
232,Влияние искусства на эмоциональное состояние,ВКонтакте,"Любознательные пользователи, интересующиеся ис...",NaN,32,extract,5,gemma-2-9b-it-Q8_0_L
472,Влияние искусства на эмоциональное состояние,ВКонтакте,"Люди всех возрастов, интересующиеся вопросами ...",Использовать хэштеги #искусство #психология #э...,32,extract,5,saiga-llama-3-q8_0
712,Влияние искусства на эмоциональное состояние,ВКонтакте,"Люди всех возрастов, интересующиеся искусством...",Использовать ссылки на известные произведения ...,32,extract,5,suzume-llama-3-8b-Q8_0


In [76]:
results[results["answer_similarity"] < 0.5]

,queries,num_message,stage,prompt,model,query_counts,answer_similarity
8,[case studies big data optimization business p...,8,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.288132
9,[future trends in big data analytics for data ...,9,web_search,0,gemma-2-9b-it-Q8_0_L,2,0.372730
132,None,12,web_search,0,saiga-llama-3-q8_0,0,0.000000
134,"[techniques of meditation for stress relief, b...",14,web_search,0,saiga-llama-3-q8_0,2,0.278385
166,None,6,web_search,1,saiga-llama-3-q8_0,0,0.000000
184,None,24,web_search,1,saiga-llama-3-q8_0,0,0.000000
254,"[techniques of meditation for stress relief, b...",14,web_search,0,suzume-llama-3-8b-Q8_0,2,0.371221


In [75]:
# min score
results[results["num_message"] == 32].iloc[0].queries

['как искусство влияет на эмоции', 'психология восприятия искусства']

In [72]:
# max score
results[results["num_message"] == 32].iloc[4].queries

['влияние искусства на эмоциональное состояние',
 'роль искусства в формировании эмоционального состояния']

In [77]:
results.prompt.unique()

array([0, 1, 2], dtype=int64)

In [78]:
num_message = 5
prompt = 0

for _, row in results[(results["num_message"] == num_message) & (results["prompt"] == prompt)][["model", "queries"]].iterrows():
    print(row.model, " : ", row["queries"])
    print()

gemma-2-9b-it-Q8_0_L  :  ['лучшие сорта томатов для открытого грунта в Сибири с описанием', 'томаты для Сибири: урожайность, сроки созревания, устойчивость к болезням']

saiga-llama-3-q8_0  :  ['сорт томатов для открытого грунта в Сибири', 'томаты для сибирского климата: лучшие сорта и особенности']

suzume-llama-3-8b-Q8_0  :  ['Лучшие сорта томатов для открытого грунта в Сибири', 'Сорты томатов для выращивания в открытом грунте в Сибири']



In [79]:
num_message = 29
prompt = 2

for _, row in results[(results["num_message"] == num_message) & (results["prompt"] == prompt)][["model", "queries"]].iterrows():
    print(row.model, " : ", row["queries"])
    print()

gemma-2-9b-it-Q8_0_L  :  ['методы развития креативности', 'упражнения для повышения креативного мышления']

saiga-llama-3-q8_0  :  ['Методы развития креативности', 'Упражнения для мыслительного процесса']

suzume-llama-3-8b-Q8_0  :  ['Как развить креативность и мыслить нестандартно?', 'Методы и упражнения для развития креативности: что работает?']



In [80]:
results.to_csv('./data/web_search_metrics.csv', index=False)

### Результаты stage web_search  

#### Промпт  
***№ 1.*** Слабый вариант, не гарантирует до 5 запросов + бывают запросы с хештегами. Были кейсы с генерацией на другом языке(англ и испанский).  
***№ 2.*** Вариант с самым простым описанием. Не хватает инструкций, запросы простые и сильно разнонаправленные. + ламает сайгу  
***№ 3.*** Лучший вариант, запросы помогают изучить тему с разных сторон и учитывают уровень целевой аудитории.  
***№ 4.*** Вариант с примером заполнения JSON сломал сайгу полностью, поэтому был исключен до тестирования.
